In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt    
from matplotlib import rcParams, cycler
from decimal import Decimal

In [ ]:
decode_for = 'stim'

In [ ]:
base_path =  '/home/rudy/Python3/cluster/'
file_path = base_path +'cnn2.2/' 
file_name = 'pval_all_sess_'+ decode_for + '.csv'

df = pd.read_csv(file_path + file_name)

In [ ]:
df.keys()

In [ ]:
only_correct_trials = True if decode_for == 'stim' else False
df = df[ (df.only_correct_trials == only_correct_trials) & (df.decode_for == decode_for)]

In [ ]:
#df

In [ ]:
#df.keys()

In [ ]:
session = np.unique(df.session)
print('list of sessions :')
print(session)


In [ ]:
#intervals = np.unique(df.interval)
print('intervals : ')
intervals = np.unique(df.interval_name)
intervals =['pre-sample', 'sample', 'delay-start', 'delay-end', 'match']
print(intervals)




In [ ]:
print('Cortex :')
cortex = np.unique(df.cortex)
cortex =['All', 'Visual', 'Prefrontal', 'Motor', 'Parietal','Somatosensory']
print(cortex)

In [ ]:
print('frequency_band :')
frequency_band =  np.unique(df.frequency_band)
frequency_band = [ 'low4high8order3', 'low8high15order3', 'low15high30order3', 'low30high80order3', 'low80high300order3']
print(frequency_band)

In [ ]:
from scipy.stats import ttest_1samp

def metrics(values, pop_mean):
    """Returns mean, pvals (1-sample t-test) and sign for array of values."""
    values = np.array(values).astype('float')
    mean = np.mean(values)
    _, p = ttest_1samp(values, pop_mean)
    sign = '***' if p < .001 else '**' if p < .01 else '*' if p < .05 else 0
    return mean, p, sign

In [ ]:
# run it again to see the color code
print('Color code for session : ')
cmap = plt.cm.rainbow
color = cmap(np.linspace(0, 1, 62))
rcParams['axes.prop_cycle'] = cycler(color=color)
matrix = np.zeros((10,len(df.session.unique())))
for i in range(10):
    for j in range(len(df.session.unique())):
        matrix[i,j] = j+1
          
plt.plot(matrix)
plt.title('Rainbow')

plt.show()

In [ ]:
print('Evolution of mean of accuracy per class \nby cortex and frequency band ')

In [ ]:
################ COLOR #######################
cm_color = 'rainbow'#'Spectral' # red (low) to blue (high)
cm_order_by = 'sess_all'#'sess_available' # 'sess_all', 'sess_available', 'acc_monkey' , 'num_electrode'
##############################################
# ['session', 'decode_for', 'only_correct_trials',
#                       'areas', 'cortex', 'elec_type',
#                       'frequency_band',
#                       'interval',
#                       'recall_macro', 'error_bar_th', 'error_bar_emp',
#                       'seed', 'n_splits',
#                       'n_test_per_class', 'data_size', 'n_chans', 'window_size'] 

if decode_for == 'stim':
    acc_random = 0.2
    ymin = 0.1
    ymax = 0.8
else :
    acc_random = 0.5
    ymin = 0.3
    ymax = 0.8

for c in cortex :
    for f in frequency_band:
        Y = [] # recall macro

        PVAL = [] # pval calculed on acc balanced 
        cm_value = [] # which color for each session
        for count, sess_no in enumerate(session):
            y = len(intervals) * [0]
            y_pval= len(intervals) * [0]

            missing_data = False
            for i in range(len(intervals)):
                interval = intervals[i]
                
                data = df[ ( df.decode_for == decode_for) & (df.cortex == c ) & (df.frequency_band == f) & (df.interval_name == interval) & (df.session == sess_no)]
                acc = data['mean_per_class_accuracy'].values
                pval = data['pval'].values
                if len(acc) !=0:
                    y[i] = acc[0]
                    y_pval[i] = pval[0]

                else :
                    missing_data = True
                    
            if not missing_data :
                
                Y.append(y)    
                PVAL.append(y_pval)
                                
                if cm_order_by == 'sess_all':
                    cm_value.append(count)
#                 elif cm_order_by =='acc_monkey':
#                     cm_value.append( data['acc_monkey'].values[0] )
#                 elif cm_order_by == 'num_electrode':
#                     cm_value.append( data['num_electrode'].values[0] )
#                 elif cm_order_by == 'num_trials':
#                     cm_value.append( data['num_trials'].values[0] )
                
            #plt.plot(intervals, y)
            
        Y = np.array(Y)
        PVAL = np.array(PVAL)
        
        
        plt.figure(figsize=(15,5))
        plt.suptitle(str(c) + ' ' + str(f))
  
        
        ## GRAPH of recall macro
        
        #print(Y.shape)
        
        data = np.transpose(Y)
        N = data.shape[1]
        #print(data.shape)
        
        if cm_color == 'Spectral':
            cmap = plt.cm.Spectral
        if cm_color == 'rainbow' :
            cmap = plt.cm.rainbow

        if cm_order_by == 'sess_available' :
            color = cmap(np.linspace(0, 1, N))
        else :
            mini = np.min(cm_value)
            maxi = np.max(cm_value)
            color = cmap( (cm_value-mini)/(maxi - mini) )
        rcParams['axes.prop_cycle'] = cycler(color=color)
        


        rcParams['axes.prop_cycle'] = cycler(color=color)

#         fig, ax = plt.subplots()
#         lines = ax.plot(data)
        
        plt.subplot(121)
        plt.plot(intervals, np.transpose(Y))
#         plt.ylim(ymin, ymax)
        plt.plot([0,4],[acc_random, acc_random],'--', color='red')
        plt.ylabel('mean_per_class_accuracy')
        plt.title('mean_per_class_accuracy')
        
        
        # BOXPLOT
        plt.subplot(122)
        

        y = np.mean(Y, axis=0)
        medianprops = dict(linestyle='-', linewidth=2.5, color='firebrick')
        plt.boxplot(Y,0, '', labels= intervals, medianprops=medianprops)
        plt.plot([1,2,3,4,5], y, 'b') # affiche la courbe de la moyenne des precisions
        plt.plot([0,6],[acc_random, acc_random],'--', color = 'red')
        
        plt.title( str(c) + ' ' + str(f))
        plt.title('Boxplot')
        #plt.ylim(ymin, ymax)
                

        #P VAL on all session
        
        pval_allsession = Y.shape[1] * [0]
        mean_precision_allsession = Y.shape[1] * [0]
        sign_pval = Y.shape[1] * [0]
    
        for i in range(Y.shape[1]):
            if decode_for == 'stim':
                pop_mean = .2
            else :
                pop_mean = .5
                
            mean_precision_allsession[i], pval_allsession[i], sign_pval[i] = metrics(Y[:,i],pop_mean)
            

        plt.show()
        
        print('null hypothese : random behavior')
        print('pval as each session would be an observation :')
        
#         line1_to_print = ''
#         line2_to_print = ''
#         line3_to_print = ''
#         for i in range(len(intervals)):
#             line1_to_print += intervals[i] + ' '
#             line2_to_print += '{:.2e}'.format(Decimal(pval_allsession[i])) + ' '
#             line3_to_print += str(sign_pval[i]) +' '
#             #print('{:.2e}'.format(Decimal(pval)))
            
#         print(line1_to_print)
#         print(line2_to_print)
#         print(line3_to_print)
        
        print('\ninterval'+ 6*' ' + 'pval'+ 6*' '+ 'sign' )
        for i in range(len(intervals)):
        
            dec = '{:.2e}'.format(Decimal(pval_allsession[i]))
            
            print(intervals[i] + ' ' * (11-len(intervals[i])) + ' : ' 
                 + dec + (10 - len(dec))*' ' 
                 + str(sign_pval[i]))
        
        
            
            
        #print(pval_allsession)
        #print('sign')
        #print(sign_pval)
        
        
        # family wise error rate
        print('\nfamily wise error rate :')
        print('according to the pval of each session')
        print('')
        print('null hypothese : random behavior')
        print('reject the null hypothese for R /(number of session) ')
        print('\n')
        tableau = np.zeros((5,3))
        
        for i in range(0,5): # for each interval
            pval = sorted(PVAL[:,i]) # pval of the interval
            # Hochberg's step-up procedure
            m = len(pval)
            for count, alpha in enumerate([0.05, 0.01, 0.001]) :
            #print('alpha :', str(alpha) )
            
#                 ## step up
#                 R = 0
#                 for k in range(len(pval)):
#                     if pval[-k-1] <= alpha /(1+k) :
#                         R = m-k # start at 1
#                         break
                        
                        
                ## step down
                R = m 
                for k in range(m):
                    if pval[k] > alpha / (m-k) :
                        R = k
                        break
                        
                tableau[i,count] = R
                #print(intervals[i] + ' : '+ str(R) + '/' + str(m) )
                
            
                
            #print('\n')
        #print('number of session : ', m)
        print('alpha :        0.05    0.01    0.001')
        
        for i in range(0,5):
            R0 = tableau[i,0]
            R1 = tableau[i,1]
            R2 = tableau[i,2]
            print(intervals[i] + ' ' * (11-len(intervals[i])) + ' :' 
                  + ' ' + (2 - len(str(int(R0)))) * ' ' + str(int(R0)) +'/'+str(int(m)) 
                  + ' '*3 + (1-len(str(int(R1)))) * ' ' + str(int(R1)) +'/'+str(int(m))
                  + ' '*3 + (1-len(str(int(R2)))) * ' ' + str(int(R2)) +'/'+str(int(m)) )
                                                                              

In [ ]:
##### boxplot frequency / cortex

In [ ]:
print('Boxplot of the results with the same scale')

In [ ]:
#decode_for = 'stim' # stim or resp

if decode_for == 'stim':
    acc_random = 0.2
    ymin = 0.1
    ymax = 0.7
else :
    acc_random = 0.5
    ymin = 0.3
    ymax = 0.7

for c in cortex :
    plt.figure(figsize=(15,5))
    plt.suptitle( str(c) )
    for count, f in enumerate(frequency_band):
        Y = [] # recall macro

        PVAL = [] # pval calculed on acc balanced 
        for sess_no in session:
            y = len(intervals) * [0]
            missing_data = False
            for i in range(len(intervals)):
                interval = intervals[i]
                
                data = df[ ( df.decode_for == decode_for) & (df.cortex == c ) & (df.frequency_band == f) & (df.interval_name == interval) & (df.session == sess_no)]
                acc = data['mean_per_class_accuracy'].values
                pval = data['pval'].values
                if len(acc) !=0:
                    y[i] = acc[0]
                    y_pval[i] = pval[0]
                else :
                    missing_data = True
                    
            if not missing_data :
                
                Y.append(y)    
                PVAL.append(y_pval)
                
            #plt.plot(intervals, y)
            
        Y = np.array(Y)
        PVAL = np.array(PVAL)
               
        # BOXPLOT
        num_subplot = 100 + len(frequency_band)*10 + count +1
        #print(num_subplot)
        plt.subplot( num_subplot )

        y = np.mean(Y, axis=0)
        medianprops = dict(linestyle='-', linewidth=2.5, color='firebrick')
        plt.boxplot(Y,0, '', labels= intervals, medianprops=medianprops)
        plt.xticks(rotation=45) # 
        plt.plot([1,2,3,4,5], y, 'b') # affiche la courbe de la moyenne des precisions
        plt.plot([0,6],[acc_random, acc_random],'--', color = 'red')
        #plt.plot([0,6],[acc_random+0.034, acc_random+0.034],'--', color ='green')
        
        plt.title( str(f) ) 
        plt.ylim(ymin, ymax)

    

    plt.show()

        
        
        


In [ ]:
#########################################""

In [ ]:
print('detailled results')
print('each dot represents a results on one session')
print('')
print('color code :')
print('pval : <0.001   < 0.01   < 0.05   >0.05')
print('code : green    orange   red      black')


In [ ]:
################ COLOR #######################
cm_order_by = 'sess_all'#'sess_available' # 'sess_available', 'acc_monkey' , 'num_electrode', 'num_trials', 'sess_all'
##############################################

#decode_for = 'stim' # stim or resp

if decode_for == 'stim':
    acc_random = 0.2
    ymin = 0.1
    ymax = 0.8
else :
    acc_random = 0.5
    ymin = 0.3
    ymax = 0.8

for c in cortex :
    for f in frequency_band :
        
        plt.figure(figsize=(15,5))
        plt.suptitle( str(c)+ ' '  + str(f), y =1.1 )

        Y = [] # recall macro
        Y_std = [] # std
        PVAL = [] # pval calculed on acc balanced 
        cm_value = [] # which color for each session
        for count, sess_no in enumerate(session):
            y = len(intervals) * [0]
            y_std = len(intervals) * [0]
            y_pval= len(intervals) * [0]
            missing_data = False


            for i in range(len(intervals)):
                interval = intervals[i]

                data = df[ ( df.decode_for == decode_for) & (df.cortex == c ) & (df.frequency_band == f) & (df.interval_name == interval) & (df.session == sess_no)]
                acc = data['mean_per_class_accuracy'].values
                acc_std = data['error_bar'].values
                pval = data['pval'].values
                if len(acc) !=0:
                    y[i] = acc[0]
                    y_std[i] = acc_std[0]
                    y_pval[i] = pval[0]
                    

                else :
                    missing_data = True

            if not missing_data :

                Y.append(y)    
                Y_std.append(y_std)
                PVAL.append(y_pval)

                if cm_order_by == 'sess_all':
                    cm_value.append(count)
                elif cm_order_by =='acc_monkey':
                    cm_value.append( data['acc_monkey'].values[0] )
                elif cm_order_by == 'num_electrode':
                    cm_value.append( data['num_electrode'].values[0] )
                elif cm_order_by == 'num_trials':
                    cm_value.append( data['num_trials'].values[0] )
                    

            #plt.plot(intervals, y)

        Y = np.array(Y)
        Y_std = np.array(Y_std)
        PVAL = np.array(PVAL)

        if cm_order_by == 'sess_available' :
             cm_value = np.linspace(0, 1, N)
        n = Y.shape[0]
        # plot
        for count, interval in enumerate(intervals):

            num_subplot = 100 + len(intervals)*10 + count +1
            #print(num_subplot)
            plt.subplot( num_subplot )    

            #plt.scatter(cm_value, Y[:, count], color ='blue')
            #plt.errorbar(cm_value, Y[:, count],yerr=2*Y_std[:,count], linestyle="None", color ='blue')
            #plt.errorbar(cm_value, Y[:, count],yerr=2*Y_std[:,count], fmt='o', color='black', ecolor='lightgray', elinewidth=3, capsize=0)
            for i in range(0,Y.shape[0]):
#                 if Y[i, count] - 2*Y_std[i,count] > acc_random : # interval de confiance à 95% ok
#                     color = 'green'
#                 elif Y[i, count] - Y_std[i,count] > acc_random : #interval de confiance à 70% ok
#                     color = 'orange'
#                 else:
#                     color ='black'
                p = PVAL[i,count] 
                color = 'green' if p < .001 else 'orange' if p < .01 else 'red' if p < .05 else 'black'
                if Y[i,count] < acc_random :
                    color = 'black'

                plt.errorbar(cm_value[i], Y[i, count],yerr=2*Y_std[i,count], fmt='o', color=color, ecolor='lightgray', elinewidth=3, capsize=0)
            plt.xlabel(cm_order_by)
            plt.ylabel('mean_per_class_accuracy')

            plt.title(interval)

            plt.plot([np.min(cm_value), np.max(cm_value)],[acc_random, acc_random],'--', color = 'red')
            #plt.plot([np.min(cm_value), np.max(cm_value)],[acc_random+ 0.04, acc_random + 0.04],'--', color = 'green')
            plt.ylim(ymin, ymax)
            

        plt.show()